In [55]:
import numpy as np
import math
from itertools import product

In [61]:
class Hopfield:
    def __init__(self, patterns):
        self.patterns = patterns
        self.n_patterns = len(patterns[0])
        self.w = None
        self.epoch = None

    def calculate_weight(self):
        temp = np.dot(self.patterns.T, self.patterns)
        np.fill_diagonal(temp, 0.0)
        self.w = temp
        return self.w

    def calculate_closest(self, a, epoch):
        activation = np.array(a)
        self.epoch = epoch
        res = 'Not Stable!'

        # initialize lists for maintaining the choices
        choices = [activation]
        diffs = [math.inf]

        for i in range(self.epoch):
            # calculate the activation
            activation = np.sign(np.dot(activation, self.w))
            choices.append(activation)

            # calculate the difference
            diff = self.n_patterns - np.count_nonzero(np.equal(a, activation))
            diffs.append(diff)

            # check whether it is stable or not
            # only check the last 2 items
            if np.count_nonzero(np.equal(choices[-1], choices[-2])) == self.n_patterns:
                if i == 0:
                    # it means that the initial input is stable
                    res = 'Stable!'
                break
        
        # calculate the details about closest one
        np_choices = np.array(choices)
        np_diffs = np.array(diffs)

        idx_closest = np.argmin(np_diffs)

        closest = np_choices[idx_closest]
        energy = -1 * np.sum(self.w * np.outer(closest, closest))
        acc = 100 * ((self.n_patterns - np_diffs[idx_closest]) / self.n_patterns)

        return closest, energy, acc, res

# A

In [62]:
# given patterns
P = np.array([
    [-1, -1, 1, -1, 1, -1, -1, 1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, 1, 1, -1, -1, 1, -1, 1]
])

# create hopfield object
h = Hopfield(P)
w = h.calculate_weight()
print(w)

[[ 0  1 -1  3  1 -1  3 -1]
 [ 1  0  1  1 -1  1  1  1]
 [-1  1  0 -1  1 -1 -1  3]
 [ 3  1 -1  0  1 -1  3 -1]
 [ 1 -1  1  1  0 -3  1  1]
 [-1  1 -1 -1 -3  0 -1 -1]
 [ 3  1 -1  3  1 -1  0 -1]
 [-1  1  3 -1  1 -1 -1  0]]


In [64]:
# Check if the given inputs are stable or not
P = np.array([
    [-1, -1, 1, -1, 1, -1, -1, 1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, 1, 1, -1, -1, 1, -1, 1],
])

for i in range(3):
    closest, energy, acc, res = h.calculate_closest(P[i], 5)
    print("\nPattern {}: {}".format(i + 1, res))


Pattern 1: Stable!

Pattern 2: Stable!

Pattern 3: Stable!


# B

In [66]:
# Check noisy patterns
P = np.array([
    [1, -1, 1, -1, 1, -1, -1, 1],
    [1, 1, -1, -1, -1, 1, -1, -1],
    [1, 1, 1, -1, 1, 1, -1, 1]
])

for i in range(3):
    closest, energy, acc, res = h.calculate_closest(P[i], 100)
    print("\nPattern {}: {}".format(i + 1, res))
    print("Closest node is {} with accuracy {}%".format(closest, acc))
    print("Energy is {}".format(energy))


Pattern 1: Not Stable!
Closest node is [-1 -1  1 -1  1 -1 -1  1] with accuracy 87.5%
Energy is -44

Pattern 2: Not Stable!
Closest node is [ 1  1 -1 -1 -1  1 -1 -1] with accuracy 100.0%
Energy is -12

Pattern 3: Not Stable!
Closest node is [-1 -1  1 -1  1  1 -1  1] with accuracy 75.0%
Energy is -32


# C

In [73]:
l = [-1, 1]
all_patterns = list(product(l, repeat=8))
stable_patterns = []

for i in range(256):
    closest, energy, acc, res = h.calculate_closest(all_patterns[i], 5)
    if(res=='Stable!'):
        stable_patterns.append(all_patterns[i])

print("{} stable patterns are found!".format(len(stable_patterns)))
for p in stable_patterns:
    print(p)

6 stable patterns are found!
(-1, -1, -1, -1, -1, 1, -1, -1)
(-1, -1, 1, -1, 1, -1, -1, 1)
(-1, 1, 1, -1, -1, 1, -1, 1)
(1, -1, -1, 1, 1, -1, 1, -1)
(1, 1, -1, 1, -1, 1, 1, -1)
(1, 1, 1, 1, 1, -1, 1, 1)
